<a href="https://colab.research.google.com/github/mshumer/gpt-prompt-engineer/blob/main/gpt_prompt_engineer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auto Prompt Engineering 
Original Notebook By Matt Shumer (https://twitter.com/mattshumer_)

Original Github repo: https://github.com/mshumer/gpt-prompt-engineer

In this notebook, we will simulate how to generate an optimal prompt/instruction for a given task.

## Installations

In [1]:
!pip install -r ../requirements.txt -qq

In [2]:
 !pip install -U langchain-community -q

## Import Libraries

In [1]:
from datasets import load_dataset, Dataset
import pandas as pd
from ast import literal_eval
import sys
from pathlib import Path

In [2]:
path_to_src_code = '../'
sys.path.append(path_to_src_code)

In [3]:
from src.utils.functions import *
from src.run_autoprompt import run_autoprompts

## Import and Preprocess Dataset

In [4]:
fiqa_eval = load_dataset("explodinggradients/fiqa", "ragas_eval")
data = fiqa_eval['baseline']

In [5]:
data.to_pandas()

,question,ground_truths,answer,contexts
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...
3,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...",\nApplying for and receiving business credit c...,[Set up a meeting with the bank that handles y...
4,401k Transfer After Business Closure,[You should probably consult an attorney. Howe...,\nIf your employer has closed and you need to ...,[The time horizon for your 401K/IRA is essenti...
5,What are the ins/outs of writing equipment pur...,[Most items used in business have to be deprec...,\nWriting equipment purchases off as business ...,[You would report it as business income on Sch...
6,Can a entrepreneur hire a self-employed busine...,[Yes. I can by all means start my own company ...,"\nYes, an entrepreneur can hire a self-employe...",[Yes. I can by all means start my own company ...
7,Intentions of Deductible Amount for Small Busi...,"[""If your sole proprietorship losses exceed al...",\nThe intention of deductible amounts for smal...,"[""Short answer, yes. But this is not done thro..."
8,How can I deposit a check made out to my busin...,[You should have a separate business account. ...,\nYou can deposit a check made out to your bus...,"[""I have checked with Bank of America, and the..."
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,\nFiling personal taxes with 1099s versus fili...,[Depends whom the 1099 was issued to. If it wa...


### Ragas

This section goes over how to call the Ragas library on the input dataset. 

In [6]:
# add directory to aws_ragas pyproject.toml file 
!pip install ragas --quiet

In [7]:
from datasets import Dataset 
import os
from ragas import evaluate
from ragas.metrics import faithfulness, answer_correctness, context_precision, answer_similarity
from langchain_community.chat_models import BedrockChat
from langchain_community.embeddings import BedrockEmbeddings

In [8]:
config = {
    "region_name": "us-east-1",  # E.g. "us-east-1"
    "model_id": 'anthropic.claude-3-haiku-20240307-v1:0',  # E.g "anthropic.claude-v2"
    "model_kwargs": {"temperature": 0.4},
}
bedrock_model = BedrockChat(
    region_name=config["region_name"],
    endpoint_url=f"https://bedrock-runtime.{config['region_name']}.amazonaws.com",
    model_id=config["model_id"],
    model_kwargs=config["model_kwargs"],
)

# init the embeddings
bedrock_embeddings = BedrockEmbeddings(
    region_name=config["region_name"],
)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(


In [9]:
score = evaluate(data,metrics=[faithfulness,answer_correctness, context_precision, answer_similarity], llm=bedrock_model,
    embeddings=bedrock_embeddings)
score.to_pandas()

passing column names as 'ground_truths' is deprecated and will be removed in the next version, please use 'ground_truth' instead. Note that `ground_truth` should be of type string and not Sequence[string] like `ground_truths`


Evaluating:   0%|          | 0/120 [00:00<?, ?it/s]

Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.
Failed to parse output. Returning None.


,question,ground_truths,answer,contexts,ground_truth,faithfulness,answer_correctness,context_precision,answer_similarity
0,How to deposit a cheque issued to an associate...,[Have the check reissued to the proper payee.J...,\nThe best way to deposit a cheque issued to a...,[Just have the associate sign the back and the...,Have the check reissued to the proper payee.Ju...,NaN,0.768410,0.0,0.740307
1,Can I send a money order from USPS as a business?,[Sure you can. You can fill in whatever you w...,"\nYes, you can send a money order from USPS as...",[Sure you can. You can fill in whatever you w...,Sure you can. You can fill in whatever you wa...,NaN,0.570801,1.0,0.871439
2,1 EIN doing business under multiple business n...,[You're confusing a lot of things here. Compan...,"\nYes, it is possible to have one EIN doing bu...",[You're confusing a lot of things here. Compan...,You're confusing a lot of things here. Company...,NaN,0.302608,1.0,0.610433
3,Applying for and receiving business credit,"[""I'm afraid the great myth of limited liabili...",\nApplying for and receiving business credit c...,[Set up a meeting with the bank that handles y...,"""I'm afraid the great myth of limited liabilit...",0.928571,0.665723,1.0,0.662891
4,401k Transfer After Business Closure,[You should probably consult an attorney. Howe...,\nIf your employer has closed and you need to ...,[The time horizon for your 401K/IRA is essenti...,You should probably consult an attorney. Howev...,0.888889,0.364961,1.0,0.259844
5,What are the ins/outs of writing equipment pur...,[Most items used in business have to be deprec...,\nWriting equipment purchases off as business ...,[You would report it as business income on Sch...,Most items used in business have to be depreci...,NaN,0.680221,1.0,0.720885
6,Can a entrepreneur hire a self-employed busine...,[Yes. I can by all means start my own company ...,"\nYes, an entrepreneur can hire a self-employe...",[Yes. I can by all means start my own company ...,Yes. I can by all means start my own company a...,1.000000,0.570512,1.0,0.645683
7,Intentions of Deductible Amount for Small Busi...,"[""If your sole proprietorship losses exceed al...",\nThe intention of deductible amounts for smal...,"[""Short answer, yes. But this is not done thro...","""If your sole proprietorship losses exceed all...",1.000000,0.631389,0.0,0.425558
8,How can I deposit a check made out to my busin...,[You should have a separate business account. ...,\nYou can deposit a check made out to your bus...,"[""I have checked with Bank of America, and the...",You should have a separate business account. M...,NaN,0.646512,1.0,0.759963
9,Filing personal with 1099s versus business s-c...,[Depends whom the 1099 was issued to. If it wa...,\nFiling personal taxes with 1099s versus fili...,[Depends whom the 1099 was issued to. If it wa...,Depends whom the 1099 was issued to. If it was...,NaN,0.579534,1.0,0.739187


In [15]:
score.to_pandas().to_csv("../data/ragas_output.csv", index=False)

## Automatic Prompt Generation

Here, automatic prompt generation is done using specific templates (available in instruction_generation_templates folder). 

In [6]:
files = [
    "one-paragraph_instruction_with_demo.txt", 
    "one-sentence_instruction_with_demo.txt", 
    "step-by-step_instruction_with_demo.txt"
]

In [7]:
run_autoprompts(auto=True, path_to_ragas_outputs="../data/ragas_output.csv")

/home/ec2-user/SageMaker/autoPromptEng/notebooks/../src/run_autoprompt.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_prompts['contexts'] = fix_prompts['contexts'].apply(literal_eval)
/home/ec2-user/SageMaker/autoPromptEng/notebooks/../src/run_autoprompt.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fix_prompts['ground_truths'] = fix_prompts['ground_truths'].apply(
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecation

9


  0%|                                          | 0/27 [00:00<?, ?it/s]

Gen 1:   Here is a one paragraph instruction on how to solve the task without mentioning any content from the context:

To complete the task, one should first analyze what the goal or desired outcome is. Then consider what different approaches or steps could be taken to work towards that outcome. Evaluate each approach based on
Gen 2:   Here is a one paragraph instruction on how to solve the task without mentioning any content from the context:

To complete the task, one should break it down into a series of logical steps. First, examine the overall goal and any conditions that must be met. Then research previous approaches to similar undert
----------


  7%|██▌                               | 2/27 [00:05<01:03,  2.53s/it]

...

Draw
Gen 1:   Here is a one sentence instruction for the task without mentioning any context:

Follow the steps necessary to accurately complete the objective.
Gen 2:   Here is a one sentence instruction for the task without mentioning any context: Follow the steps necessary to complete the objective.
----------


100%|█████████████████████████████████| 27/27 [02:48<00:00,  6.85s/it]

Draw
Gen 1:   Here is a step-by-step instruction on how to solve the given task without mentioning any content from the context:

1. Read the full task and ensure you understand what is being asked. 

2. Break down the task into individual steps or components that need to be completed.
Gen 2:   Here is a step-by-step instruction on how to solve the task without mentioning any content from the context:

1. Carefully read and analyze the full context to understand all aspects of the task. 

2. Identify the overarching goal or objective implied in the task
----------


100%|█████████████████████████████████| 27/27 [02:54<00:00,  6.48s/it]

Draw


Below is the output of the best three instructions for each question in the dataset. 

In [8]:
new_prompts = pd.read_csv('../data/new_prompts.csv')

In [9]:
new_prompts.head()

,question,ground_truths,answer,contexts,ground_truth,faithfulness,answer_correctness,context_precision,answer_similarity,best_prompt,second_best_prompt,third_best_prompt
0,How to deposit a cheque issued to an associate...,"[""Have the check reissued to the proper payee....",\nThe best way to deposit a cheque issued to a...,['Just have the associate sign the back and th...,Have the check reissued to the proper payee.Ju...,NaN,0.768410,0.0,0.740307,Examine the question and provided background t...,"Read the question and context provided, and ge...",Reflect on the question posed and context supp...
1,1 EIN doing business under multiple business n...,"[""You're confusing a lot of things here. Compa...","\nYes, it is possible to have one EIN doing bu...",['You\'re confusing a lot of things here. Comp...,You're confusing a lot of things here. Company...,NaN,0.302608,1.0,0.610433,"Carefully review the question and context, and...",Read the provided context and examone the ques...,Focus on comprehending the relationship betwee...
2,401k Transfer After Business Closure,"[""You should probably consult an attorney. How...",\nIf your employer has closed and you need to ...,['The time horizon for your 401K/IRA is essent...,You should probably consult an attorney. Howev...,0.888889,0.364961,1.0,0.259844,Analyze the question and context input to dedu...,Carefully read the provided question and conte...,Contemplate the question and context thoroughl...
3,Intentions of Deductible Amount for Small Busi...,"['""If your sole proprietorship losses exceed a...",\nThe intention of deductible amounts for smal...,"['""Short answer, yes. But this is not done thr...","""If your sole proprietorship losses exceed all...",1.000000,0.631389,0.0,0.425558,Focus on comprehending the core issue or topic...,"Read and comprehend the question and context, ...",Read the question and context carefully to und...
4,Using credit card points to pay for tax deduct...,"['""For simplicity, let\'s start by just consid...",\nUsing credit card points to pay for tax dedu...,"['""For simplicity, let\'s start by just consid...","""For simplicity, let's start by just consideri...",0.833333,0.469786,1.0,0.820322,"Analyze the question and context, and synthesi...",You have been given a question to answer along...,A question and supporting details have been sh...


Below are the prompts/instructions mapped to ids. 

In [10]:
prompt_ids = pd.read_csv('../data/prompt_ids.csv')

In [11]:
prompt_ids.head()

,Id,Prompts
0,0,Examine the question and provided background t...
1,1,"Carefully review the question and context, and..."
2,2,Analyze the question and context input to dedu...
3,3,Focus on comprehending the core issue or topic...
4,4,"Analyze the question and context, and synthesi..."
